In [1]:
import pandas as pd

In [2]:
# HN16_ALL : 가구조사, 건강면접조사, 건강형태조사, 검진조사, 영양조사
# NH16_24RC : 식품섭취조사
# HN16_FFQ : 식품섭취빈도조사

df_ALL = pd.read_sas("./HN16_ALL.sas7bdat") 
#df_24RC = pd.read_sas("./HN16_24RC.sas7bdat")
df_ffq = pd.read_sas("./hn16_ffq.sas7bdat")
# df_24RC_code = pd.read_excel("./HN16_24RC(식품_음식코드).xlsx")

In [10]:
df = pd.merge(df_ALL, df_ffq, on= 'ID', how='left')

In [11]:
df.shape

(8150, 1201)

In [12]:
print(df.head(3))

          mod_d_x             ID     ID_fam_x  year_x  region_x  town_t_x  \
0  b'2018.02.01.'  b'A651172801'  b'A6511728'  2016.0       1.0       1.0   
1  b'2018.02.01.'  b'A651172802'  b'A6511728'  2016.0       1.0       1.0   
2  b'2018.02.01.'  b'A651183001'  b'A6511830'  2016.0       1.0       1.0   

   apt_t_x    psu_x  sex_x  age_x    ...          FQ_FE        FQ_NA  \
0      2.0  b'A651'    1.0   61.0    ...            NaN          NaN   
1      2.0  b'A651'    2.0   57.0    ...            NaN          NaN   
2      2.0  b'A651'    2.0   41.0    ...      12.879218  3671.647791   

          FQ_K       FQ_VA   FQ_RETIN     FQ_CAROT     FQ_B1    FQ_B2  \
0          NaN         NaN        NaN          NaN       NaN      NaN   
1          NaN         NaN        NaN          NaN       NaN      NaN   
2  2226.426572  459.311661  84.990392  2158.129699  1.628015  1.13383   

     FQ_NIAC    FQ_VITC  
0        NaN        NaN  
1        NaN        NaN  
2  13.896055  79.150355  

[3 r